# 朴素贝叶斯分类器

作者：周阳

主要任务：

* 分类文件整合
* 基于朴素贝叶斯的概率统计
* 每个类别构建词表（20个log词表）
* 通过查表完成测试分类

## 通过文件夹分类文件并且整合

In [17]:
import random as ran
import os
import numpy as np

In [18]:
classis = {0:'/home/blueberry/data/datamining/20news-18828/alt.atheism',
           1:'/home/blueberry/data/datamining/20news-18828/comp.graphics',
           2:'/home/blueberry/data/datamining/20news-18828/comp.os.ms-windows.misc',
           3:'/home/blueberry/data/datamining/20news-18828/comp.sys.ibm.pc.hardware',
           
           4:'/home/blueberry/data/datamining/20news-18828/comp.sys.mac.hardware',
           5:'/home/blueberry/data/datamining/20news-18828/comp.windows.x',
           6:'/home/blueberry/data/datamining/20news-18828/misc.forsale',
           7:'/home/blueberry/data/datamining/20news-18828/rec.autos',
           
           8:'/home/blueberry/data/datamining/20news-18828/rec.motorcycles',
           9:'/home/blueberry/data/datamining/20news-18828/rec.sport.baseball',
           10:'/home/blueberry/data/datamining/20news-18828/rec.sport.hockey',
           11:'/home/blueberry/data/datamining/20news-18828/sci.crypt',
           
           12:'/home/blueberry/data/datamining/20news-18828/sci.electronics',
           13:'/home/blueberry/data/datamining/20news-18828/sci.med',
           14:'/home/blueberry/data/datamining/20news-18828/sci.space',
           15:'/home/blueberry/data/datamining/20news-18828/soc.religion.christian',
           
           16:'/home/blueberry/data/datamining/20news-18828/talk.politics.guns',
           17:'/home/blueberry/data/datamining/20news-18828/talk.politics.mideast',
           18:'/home/blueberry/data/datamining/20news-18828/talk.politics.misc',
           19:'/home/blueberry/data/datamining/20news-18828/talk.religion.misc'
          }
save_path = '/home/blueberry/data/datamining/class_with_lable.npy'

In [7]:
def traverse(f,result):
    
    fs = os.listdir(f)
    for f1 in fs:
        tmp_path = os.path.join(f,f1)
        if not os.path.isdir(tmp_path):
            result.append(tmp_path)
        else:
            print(tmp_path)
            traverse(tmp_path,result)

###### 注意这里使用的_pre文件是经过上次实验已经进行过去停用词，符号，获得词干等操作的文件结果

In [13]:
keys = list(classis.keys())
text_with_lable = []
for k in keys:
    cls_path = []
    traverse(classis[k],cls_path)
    for path in cls_path:
        if path[-4:]=='_pre':
            with open(path,'r+') as f :
                lines = f.readlines()
                lines = [line.strip('\n') for line in lines]
                lines.insert(0,k)##类别在当前list的第一个位置
            text_with_lable.append(lines)

In [14]:
text_with_lable[:2]

[[0,
  'liveseysolntzewpdsgicom',
  'jon',
  'livesey',
  'subject',
  'cruel',
  'polit',
  'atheist',
  'articl',
  'aprbmerhbnrca',
  'dgrahambmersbnrca',
  'dougla',
  'graham',
  'write',
  'articl',
  'qnedmafidoasdsgicom',
  'liveseysolntzewpdsgicom',
  'jon',
  'livesey',
  'write',
  'articl',
  'qlmdinngapcaltechedu',
  'keithccocaltechedu',
  'keith',
  'allan',
  'schneider',
  'write',
  'spent',
  'quit',
  'bit',
  'time',
  'word',
  'constitut',
  'realis',
  'wide',
  'held',
  'belief',
  'america',
  'fact',
  'claus',
  'cruel',
  'unusu',
  'punish',
  'like',
  'lot',
  'rest',
  'lift',
  'english',
  'bill',
  'right',
  'accord',
  'jerri',
  'mander',
  'absenc',
  'sacr',
  'good',
  'book',
  'btw',
  'great',
  'bind',
  'law',
  'iroquoi',
  'confederaci',
  'also',
  'play',
  'signific',
  'role',
  'model',
  'us',
  'constitut',
  'furthermor',
  'appar',
  'marx',
  'engel',
  'strong',
  'influenc',
  'studi',
  'iroquoi',
  'societi',
  'use',
  'p

#### 乱序

In [16]:
ran.shuffle(text_with_lable)
text_with_lable[:2]

[[1,
  'broehlsuneeuwaterlooca',
  'berni',
  'roehl',
  'subject',
  'optim',
  'project',
  'articl',
  'suatinnqsnonamesnerdcufledu',
  'lionessufccufledu',
  'write',
  'threed',
  'librari',
  'doe',
  'lot',
  'project',
  'duh',
  'current',
  'project',
  'object',
  'vertic',
  'per',
  'triangl',
  'basi',
  'grossli',
  'ineffici',
  'object',
  'display',
  'optim',
  'project',
  'one',
  'time',
  'onc',
  'vertex',
  'object',
  'whose',
  'zextent',
  'intersect',
  'hither',
  'plane',
  'cant',
  'benefit',
  'becaus',
  'new',
  'vertic',
  'must',
  'creat',
  'dure',
  'zclip',
  'anyon',
  'ani',
  'better',
  'idea',
  'yes',
  'keep',
  'vertic',
  'array',
  'polygon',
  'triangl',
  'okay',
  'nside',
  'polygon',
  'slight',
  'effici',
  'store',
  'indic',
  'array',
  'vertic',
  'compris',
  'set',
  'flag',
  'vertex',
  'transform',
  'dont',
  'transform',
  'ani',
  'vertex',
  'once;',
  'also',
  'backfac',
  'elimin',
  'befor',
  'process',
  'pol

#### 保存
通过npy文件序列化保存

In [19]:
np.save(save_path,text_with_lable)

In [ ]:
vocab_list =  [{},{},{},{},{},{},{},{},{},{}]